In [ ]:
pip install pandas requests ipaddress


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
from time import sleep
import requests
import ipaddress

In [5]:
API_KEY = "8045c906b36a05f803006d44442226215d311f0abec07fd89e65d5b6b9eb4b38b23b05f321afc62b"  # Replace with your real API key
CSV_INPUT_PATH = "important.csv"
CSV_OUTPUT_PATH = "threat_report.csv"
MAX_AGE_DAYS = 90
API_URL = "https://api.abuseipdb.com/api/v2/check"

In [6]:
df = pd.read_csv(CSV_INPUT_PATH)

In [7]:
df

,bytes_in,bytes_out,creation_time,end_time,src_ip,src_ip_country_code,protocol,response.code,dst_port,dst_ip,rule_names,observation_name,source.meta,source.name,time,detection_types
0,5602,12990,2024-04-25T23:00:00Z,2024-04-25T23:10:00Z,147.161.161.82,AE,HTTPS,200,443,10.138.69.97,Suspicious Web Traffic,Adversary Infrastructure Interaction,AWS_VPC_Flow,prod_webserver,2024-04-25T23:00:00Z,waf_rule
1,30912,18186,2024-04-25T23:00:00Z,2024-04-25T23:10:00Z,165.225.33.6,US,HTTPS,200,443,10.138.69.97,Suspicious Web Traffic,Adversary Infrastructure Interaction,AWS_VPC_Flow,prod_webserver,2024-04-25T23:00:00Z,waf_rule
2,28506,13468,2024-04-25T23:00:00Z,2024-04-25T23:10:00Z,165.225.212.255,CA,HTTPS,200,443,10.138.69.97,Suspicious Web Traffic,Adversary Infrastructure Interaction,AWS_VPC_Flow,prod_webserver,2024-04-25T23:00:00Z,waf_rule
3,30546,14278,2024-04-25T23:00:00Z,2024-04-25T23:10:00Z,136.226.64.114,US,HTTPS,200,443,10.138.69.97,Suspicious Web Traffic,Adversary Infrastructure Interaction,AWS_VPC_Flow,prod_webserver,2024-04-25T23:00:00Z,waf_rule
4,6526,13892,2024-04-25T23:00:00Z,2024-04-25T23:10:00Z,165.225.240.79,NL,HTTPS,200,443,10.138.69.97,Suspicious Web Traffic,Adversary Infrastructure Interaction,AWS_VPC_Flow,prod_webserver,2024-04-25T23:00:00Z,waf_rule
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,41336,13180,2024-04-26T09:50:00Z,2024-04-26T10:00:00Z,136.226.77.103,CA,HTTPS,200,443,10.138.69.97,Suspicious Web Traffic,Adversary Infrastructure Interaction,AWS_VPC_Flow,prod_webserver,2024-04-26T09:50:00Z,waf_rule
278,3638,3190,2024-04-26T09:50:00Z,2024-04-26T10:00:00Z,165.225.26.101,DE,HTTPS,200,443,10.138.69.97,Suspicious Web Traffic,Adversary Infrastructure Interaction,AWS_VPC_Flow,prod_webserver,2024-04-26T09:50:00Z,waf_rule
279,25207794,1561220,2024-04-26T09:50:00Z,2024-04-26T10:00:00Z,155.91.45.242,US,HTTPS,200,443,10.138.69.97,Suspicious Web Traffic,Adversary Infrastructure Interaction,AWS_VPC_Flow,prod_webserver,2024-04-26T09:50:00Z,waf_rule
280,5736,12114,2024-04-26T09:50:00Z,2024-04-26T10:00:00Z,165.225.209.4,CA,HTTPS,200,443,10.138.69.97,Suspicious Web Traffic,Adversary Infrastructure Interaction,AWS_VPC_Flow,prod_webserver,2024-04-26T09:50:00Z,waf_rule


In [8]:
all_ips = pd.concat([df["src_ip"], df["dst_ip"]]).dropna().unique()


In [9]:
def is_valid_ip(ip):
    try:
        ipaddress.ip_address(ip)
        return True
    except ValueError:
        return False

In [10]:
valid_ips=[ip for ip in all_ips if is_valid_ip(ip)]

In [11]:
def check_ip(ip):
    headers = {
        "Accept": "application/json",
        "Key": API_KEY
    }
    params = {
        "ipAddress": ip,
        "maxAgeInDays": str(MAX_AGE_DAYS)
    }
    try:
        response = requests.get(API_URL, headers=headers, params=params)
        data = response.json()["data"]
        score = data["abuseConfidenceScore"]
        category = "High Risk" if score >= 75 else "Medium Risk" if score >= 50 else "Low Risk"
        return {
            "ip": ip,
            "score": score,
            "category": category,
            "totalReports": data.get("totalReports", 0),
            "country": data.get("countryCode", "N/A"),
            "lastReported": data.get("lastReportedAt", "N/A")
        }
    except Exception as e:
        return {
            "ip": ip,
            "score": "Error",
            "category": "Error",
            "totalReports": "Error",
            "country": "Error",
            "lastReported": str(e)
        }

In [ ]:
results = []
for ip in valid_ips:
    result = check_ip(ip)
    results.append(result)
    print(f"Checked {ip} → {result['category']} ({result['score']})")
    sleep(1.5) 


Checked 147.161.161.82 → Low Risk (0)
Checked 165.225.33.6 → Low Risk (0)
Checked 165.225.212.255 → Low Risk (0)
Checked 136.226.64.114 → Low Risk (0)
Checked 165.225.240.79 → Low Risk (0)
Checked 136.226.77.103 → Low Risk (5)
Checked 165.225.26.101 → Low Risk (0)
Checked 155.91.45.242 → Low Risk (0)
Checked 165.225.209.4 → Low Risk (0)
Checked 147.161.131.1 → Low Risk (0)
Checked 136.226.67.101 → Low Risk (0)
Checked 94.188.248.74 → Low Risk (0)
Checked 165.225.213.7 → Low Risk (0)
Checked 165.225.8.79 → Low Risk (0)
Checked 136.226.80.97 → Low Risk (0)
Checked 192.241.230.19 → Low Risk (0)
Checked 198.235.24.81 → Low Risk (0)
Checked 65.49.1.69 → High Risk (100)
Checked 65.49.1.72 → High Risk (100)
Checked 65.49.1.94 → High Risk (100)
Checked 65.49.1.104 → High Risk (100)
Checked 65.49.1.97 → High Risk (100)
Checked 65.49.1.99 → High Risk (100)
Checked 65.49.1.76 → High Risk (100)
Checked 65.49.1.96 → High Risk (100)
Checked 65.49.1.95 → High Risk (100)
Checked 65.49.1.74 → High Risk

In [13]:
result_df = pd.DataFrame(results)
result_df.to_csv(CSV_OUTPUT_PATH, index=False)
print(f"\n✅ Threat report saved to {CSV_OUTPUT_PATH}")


✅ Threat report saved to threat_report.csv
